## <a id='toc1_1_'></a>[__Пример работы класса `DirectoryScanner`__](#toc0_)

**Содержание**<a id='toc0_'></a>
- [__Пример работы класса `DirectoryScanner`__](#toc1_1_)
    - [__Постановка задачи__](#toc1_1_1_)
    - [__Источники текстов__](#toc1_1_2_)
    - [__Соответствие между расширением файла и обработчиком__](#toc1_1_3_)
    - [__Назначение и параметризация обработчика__](#toc1_1_4_)
    - [__Инициализация объекта класса `DirectoryScanner`__](#toc1_1_5_)
    - [__Отчет писателя `HtmlReporter`__](#toc1_1_6_)
    - [__Публичный интерфейс класса `DirectoryScanner`__](#toc1_1_7_)
    - [__Создание своего обработчика__](#toc1_1_8_)
      - [__Обработчик извлекает из файла текст и возвращает его__](#toc1_1_8_1_)
      - [__Обработчик распаковывает файл в новую папку и возвращает путь к ней__](#toc1_1_8_2_)
    - [__Стратегия резки текста__](#toc1_1_9_)
    - [__Писатель отчета__](#toc1_1_10_)

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

***
### <a id='toc1_1_1_'></a>[__Постановка задачи__](#toc0_)

* Дано дерево папок произвольной вложенности. Папки содержат файлы различных расширений (документы, таблицы, изображения, архивы, почтовые сообщения и т.д.). Имея заданные ключевые слова и/или регулярные выражения, найти и свести в отчет фрагменты текстов из файлов, которые содержат искомую информацию.

* Решить задачу в соответствии с _принципом открытости/закрытости_: готовый код должен быть закрыт для модификации, но открыт для расширения. Предоставить пользователю возможность создавать свои обработчики файлов, параметризировать их и назначать на работу с расширениями по выбору.

***
### <a id='toc1_1_2_'></a>[__Источники текстов__](#toc0_)

В одной папке с ноутбуком находится архив `demo files.7z` с 6 файлами, содержащими тексты на русском языке:

In [ ]:
"""
sberpunk/examples/DirectoryScanner/demo files.7z/
├─not a docx file.docx
├─Ленинград - БСЭ.docx
├─Летний отдых россиян 1.png
├─Летний отдых россиян 2.jpeg
├─Трудовой договор.pdf
└─Туризм в России.xlsx
"""

***
### <a id='toc1_1_3_'></a>[__Соответствие между расширением файла и обработчиком__](#toc0_)

Поддерживаемые расширения файлов и соответствующие имена классов обработчиков, назначенных на то или иное расширение, находятся в атрибуте класса `DirectoryScanner.map`:

In [2]:
from sberpunk.text.directory_scanner import *

# __all__ = [
#     'DirectoryScanner',
#     'ImgTesseractHandler',
#     'HandlerBaseClass',
#     'BatchingStrategyBaseClass',
#     'ReporterBaseClass'
# ]

In [3]:
DirectoryScanner.map

{
    "7z": "ArchiveHandler",
    "csv": "AnyTextFileHandler",
    "doc": "DocxHandler",
    "docx": "DocxHandler",
    "jpg": "ImgTesseractHandler",
    "json": "AnyTextFileHandler",
    "msg": "MsgHandler",
    "pdf": "PdfHandler",
    "png": "ImgTesseractHandler",
    "pptx": "PptxHandler",
    "py": "AnyTextFileHandler",
    "rtf": "RtfHandler",
    "txt": "AnyTextFileHandler",
    "xls": "XlsHandler",
    "xlsx": "XlsxHandler",
    "zip": "ZipHandler",
}

Файлы с расширениями, не зарегистрированными в `DirectoryScanner.map`, сканером игнорируются.

***
### <a id='toc1_1_4_'></a>[__Назначение и параметризация обработчика__](#toc0_)

Из `DirectoryScanner.map` видно, что за работу с изображениями, расширения `png` и `jpg`, отвечает обработчик `ImgTesseractHandler`. Однако в примере есть файл с расширением `jpeg`. Обработчик на расширение назначается (или _расширение регистрируется_) методом класса `DirectoryScanner.update_map`:

In [4]:
DirectoryScanner.update_map({'jpeg': ImgTesseractHandler})
DirectoryScanner.map

{
    "7z": "ArchiveHandler",
    "csv": "AnyTextFileHandler",
    "doc": "DocxHandler",
    "docx": "DocxHandler",
    "jpeg": "ImgTesseractHandler",
    "jpg": "ImgTesseractHandler",
    "json": "AnyTextFileHandler",
    "msg": "MsgHandler",
    "pdf": "PdfHandler",
    "png": "ImgTesseractHandler",
    "pptx": "PptxHandler",
    "py": "AnyTextFileHandler",
    "rtf": "RtfHandler",
    "txt": "AnyTextFileHandler",
    "xls": "XlsHandler",
    "xlsx": "XlsxHandler",
    "zip": "ZipHandler",
}

`ImgTesseractHandler` необходимо параметризировать:
* указать путь к `tesseract.exe` на своей машине;
* указать язык распознавания текста

In [5]:
handler_kwargs = {
    ImgTesseractHandler: {
        # заменить на путь на своей машине
        'tesseract_cmd': 'C:/Program Files/Tesseract-OCR/tesseract.exe',
        # установку языков, отличных от английского, см. по ссылке ниже
        'tesseract_lang': 'rus'
    }
}

* [__Installing additional language packs__](https://ocrmypdf.readthedocs.io/en/latest/languages.html#installing-additional-language-packs)
* [__Python-tesseract is a python wrapper for Google's Tesseract-OCR__](https://pypi.org/project/pytesseract/)

***
### <a id='toc1_1_5_'></a>[__Инициализация объекта класса `DirectoryScanner`__](#toc0_)

Параметры:

* `keywords` — ключевые слова и/или регулярные выражения для поиска в текстах. Список строк `list[str]`.

(далее только именованные аргументы)

* `source` — локация файлов-источников. Папка с файлами или одиночный файл (например, архив с файлами). По умолчанию текущая рабочая папка. `str | pathlib.Path`.

* `report_dir` — папка для записи отчета. По умолчанию текущая рабочая папка. `str | pathlib.Path`.

* `ignore_extensions` — расширения файлов, пропускаемых обработкой даже если расширение зарегистрировано в `DirectoryScanner.map`. Множество строк `set[str]`.

* `batch_strategy` — стратегия резки текста на батчи. В данный момент реализована единственная стратегия, `LinewiseBatchingStrategy`, потомок абстрактного класса `BatchingStrategyBaseClass`. Объект `LinewiseBatchingStrategy` является значением по умолчанию данного аргумента. Реализацию `LinewiseBatchingStrategy` и как написать свою стратегию, см. в [__Стратегия резки текста__](#toc1_1_9_).

* `handler_kwargs` — именованные аргументы для параметризации соответствующих обработчиков. Структура по образцу одноименного словаря `handler_kwargs` из [__Назначение и параметризация обработчика__](#toc1_1_4_).

* `reporter` — писатель отчета. В данный момент реализован единственный писатель, `HtmlReporter`, потомок абстрактного класса `ReporterBaseClass`. Объект `HtmlReporter` является значением по умолчанию данного аргумента. Как сделать свой писатель, см. в [__Писатель отчета__](#toc1_1_10_).

Мы готовы инициализировать сканер. Будем искать в файлах фрагменты текста, содержащие заданные ниже 3 слова со всеми возможными продолжениями (суффиксы, окончания — вплоть до и не включая следующий за словом пробел):

In [6]:
scanner = DirectoryScanner(
    keywords=[word + r'\w*' for word in ('график', 'отдых', 'ущерб')],
    source='demo files.7z',
    handler_kwargs=handler_kwargs
)

INFO patool: Extracting \\?\D:\git\sberpunk\src\sberpunk\examples\DirectoryScanner\demo files.7z ...
INFO patool: running "C:\Program Files\7-Zip\7z.EXE" x "-o\\?\D:\git\sberpunk\src\sberpunk\examples\DirectoryScanner\demo files" -- "\\?\D:\git\sberpunk\src\sberpunk\examples\DirectoryScanner\demo files.7z"
INFO patool:     with input=
INFO patool: ... \\?\D:\git\sberpunk\src\sberpunk\examples\DirectoryScanner\demo files.7z extracted to `\\?\D:\git\sberpunk\src\sberpunk\examples\DirectoryScanner\demo files'.


Здесь мы видим:
* Отчет `patool` о распаковке архива `demo files.7z`;
* Уведомление пользователя о том, что файл `not a docx file.docx` фактически не является архивом. И это действительно так. Это переименованный текстовый файл, добавленный в пример для демонстрации. Вспомним, что файлы `docx` и `xlsx` являются архивами.

***
### <a id='toc1_1_6_'></a>[__Отчет писателя `HtmlReporter`__](#toc0_)

Рассмотрим получившийся отчет. Здесь он для наглядности представлен средствами `IPython`. В реальной практике удобнее, конечно, просто открыть отчет в браузере.

* Клик на ссылке в оглавлении _Table of Contents_ переведет в начало соответствующего результата поиска в _Search Results_. Запись в оглавлении (и, соответственно, в результатах) тем выше, чем больше релевантных фрагментов текста было обнаружено в данном документе.

* В таблице мы видим найденные ключевые слова и соответствующий нумерованный фрагмент текста, например, `[__batch_id='00003'__]`. Если контекста, который задает фрагмент, читателю недостаточно (либо представление текста в ячейке таблицы не удобно для чтения), клик на результате поиска откроет разрезанную на фрагменты полную текстовую версию соответствующего документа — например, ее можно открыть в новой вкладке браузера.

* По какому правилу стратегия разрезает текст на батчи токенов, из которых (пере)собираются фрагменты текста, см. в [__Стратегия резки текста__](#toc1_1_9_).

In [7]:
from IPython.display import display_html

with open('demo report.html', encoding='utf-8') as f:
    report = f.read()

display_html(report, raw=True)

keywords,batch
"[отдых, отдыха]","[__batch_id='00003'__]\n2.1.4. Обеспечение рабочего места оборудованием, инструментами, технической \nдокументацией и иными средствами, необходимыми для исполнения им трудовых \nобязанностей.\n2.1.5. Своевременную и в полном объеме выплату заработной платы в соответствии со \nсвоей квалификацией, сложностью труда, количеством и качеством выполненной работы.\n2.1.6. Отдых, то есть соблюдение ежедневной продолжительности рабочего времени, \nпредоставление перерывов для отдыха и питания, еженедельных выходных дней, \nоплачиваемых ежегодных отпусков в соответствии с настоящим трудовым договором и \nтрудовым законодательством РФ.\n2.1.7. Обязательное государственное социальное страхование в порядке и на \nусловиях, установленных действующим законодательством РФ, на период действия \nнастоящего трудового договора.\n2.1.8. Осуществление иных прав, предусмотренных трудовым законодательством РФ, \nПравилами внутреннего трудового распорядка и иными локальными нормативными \nактами.\n2.2. Работник обязан:\n2.2.1. Добросовестно исполнять трудовую функцию, соответствующую должности \nменеджера по персоналу, закрепленную в должностной инструкции (Приложение N 1),"
[ущерба],"[__batch_id='00005'__]\n2.2.6. Бережно относиться к имуществу Работодателя (в том числе к имуществу \nтретьих лиц, находящемуся у работодателя, если работодатель несет ответственность за \nсохранность этого имущества) и других работников и при необходимости принимать меры \nдля предотвращения ущерба имуществу.\n2.3. Не включение в трудовой договор каких-либо из прав и (или) обязанностей \nработника, установленных трудовым законодательством и иными нормативными \nправовыми актами, содержащими нормы трудового права, локальными нормативными \nактами, не может рассматриваться как отказ от реализации этих прав или исполнения этих \nобязанностей.\n3. Права и обязанности Работодателя\n3.1. Работодатель имеет право:\n3.1.1. Изменять и расторгать трудовой договор с Работником в порядке и на условиях, \nкоторые установлены Трудовым кодексом РФ, иными федеральными законами.\n3.1.2. Требовать от Работника исполнения им трудовых обязанностей и бережного \nотношения к имуществу Работодателя и других работников, соблюдения Правил \nвнутреннего трудового распорядка и иных локальных нормативных актов, трудовой \nдисциплины,\n производственной санитарии и \nпротивопожарной защиты."
[отдыха],"[__batch_id='00008'__]\n3.2.9. Возмещать вред, причиненный Работнику в связи с исполнением им трудовых \nобязанностей, а также компенсировать моральный вред в порядке и на условиях, которые \nустановлены Трудовым кодексом РФ, другими федеральными законами и иными \nнормативными правовыми актами РФ.\n3.2.10. Вести на Работника трудовую книжку в соответствии с законодательством \nРоссийской Федерации.\n3.2.11. Исполнять иные обязанности, предусмотренные трудовым законодательством, \nв том числе законодательством о специальной оценке условий труда, и иными \nнормативными правовыми актами, содержащими нормы трудового права, соглашениями, \nлокальными нормативными актами и настоящим трудовым договором.\n4. Рабочее время и время отдыха\n4.1. Работнику устанавливается нормальная продолжительность рабочего времени - \n40 часов в неделю.\n4.2. Работнику устанавливается следующий режим рабочего времени:\n- пятидневная рабочая неделя с двумя выходными днями (суббота и воскресенье);\n- продолжительность ежедневной работы - 8 часов;\n- начало работы - 09.00, окончание работы - 18.00;\n- перерыв для отдыха и питания - 1 час (с 13.00 до 14.00)."
"[графике, графиком]","[__batch_id='00009'__]\n4.2.1. Работодатель вправе привлекать Работника к работе в выходные и нерабочие \nпраздничные дни, а также к сверхурочной работе в порядке и на условиях, установленных \nтрудовым законодательством.\n4.3.\n Работнику предоставляется \nежегодный оплачиваемый отпуск \nпродолжительностью 28 календарных дней.\n4.3.1. Право на использование отпуска за первый год работы возникает у Работника \nпо истеч

***
### <a id='toc1_1_7_'></a>[__Публичный интерфейс класса `DirectoryScanner`__](#toc0_)

In [8]:
from sberpunk import public_api
public_api(scanner)

['batching_strategy',
 'counter',
 'handler_kwargs',
 'ignore_extensions',
 'keywords',
 'map',
 'records',
 'report_dir',
 'reporter',
 'restore_map_defaults',
 'source',
 'unique_files_seen',
 'update_map']

Разберем оставшиеся публичные члены интерфейса.

* `unique_files_seen` — число уникальных файлов, встреченных сканером:

In [9]:
# сам архив как файл + 6 файлов внутри него
scanner.unique_files_seen

7

* `counter` — счетчик найденных ключевых слов по всем документам в порядке убывания числа вхождений:

In [10]:
scanner.counter

Counter({'отдыха': 6,
         'ущерб': 2,
         'отдых': 2,
         'ущерба': 2,
         'графиков': 1,
         'график': 1,
         'графиком': 1,
         'графике': 1,
         'отдыхавших': 1})

* `restore_map_defaults` — см. раздел [__Создание своего обработчика__](#toc1_1_8_).

* `records` — атрибут, в котором консолидируются все результаты поиска. Это словарь, в котором ключи — полные пути до текстовых версий документов, значения — списки объектов служебного класса `Record` с атрибутами `keywords` и `batch`. `batch` на самом деле строка. Список же строк, который мы видим ниже, — это лишь свертка `textwrap` для компактного удобочитаемого формального и неформального представлений.
<br>Выбор такой структуры обусловлен тем, что список словарей удобно трансформировать в датафрейм.
<br>`records` это тот объект, запись которого на диск обязан реализовать любой наследующийся от базового класса писатель отчета. Подробнее в [__Писатель отчета__](#toc1_1_10_).

In [11]:
scanner.records

{'D:\\git\\sberpunk\\src\\sberpunk\\examples\\DirectoryScanner\\demo files\\Трудовой договор.pdf.txt': [{
      "keywords": ["отдых", "отдыха"],
      "batch": [
          "[__batch_id='00003'__] 2.1.4.   Обеспечение   рабочего",
          "места   оборудованием,   инструментами,   технической",
          "документацией   и   иными   средствами,   необходимыми   для",
          "исполнения   им   трудовых  обязанностей. 2.1.5.",
          "Своевременную и в полном объеме выплату заработной платы в",
          "соответствии со  своей квалификацией, сложностью труда,",
          "количеством и качеством выполненной работы. 2.1.6. Отдых, то",
          "есть соблюдение ежедневной продолжительности рабочего",
          "времени,  предоставление   перерывов   для   отдыха   и",
          "питания,   еженедельных   выходных   дней,  оплачиваемых",
          "ежегодных отпусков в соответствии с настоящим трудовым",
          "договором и  трудовым законодательством РФ. 2.1.7.",
          "Обя

***
### <a id='toc1_1_8_'></a>[__Создание своего обработчика__](#toc0_)

Обработчик обязан быть потомком базового абстрактного класса `HandlerBaseClass` и переопределять статический метод `extract_text`, который принимает путь к файлу, объект `pathlib.Path`.

In [12]:
from pathlib import Path

***
#### <a id='toc1_1_8_1_'></a>[__Обработчик извлекает из файла текст и возвращает его__](#toc0_)

In [13]:
class MyCustomFooHandler(HandlerBaseClass):
    """
    FOO file handler.
    """

    bar = None
    baz = None

    @staticmethod
    @HandlerBaseClass.write_batches
    def extract_text(path: Path) -> str:
        ...

В этом сценарии метод `extract_text` __должен__ быть декорирован `@HandlerBaseClass.write_batches`.

***
#### <a id='toc1_1_8_2_'></a>[__Обработчик распаковывает файл в новую папку и возвращает путь к ней__](#toc0_)

Так работают распаковщики архивов и обработчик почтовых сообщений `msg`: создается новая папка, в которую пишется текстовый файл с телом письма и выгружаются прикрепленные к письму файлы. Далее обработка проваливается в папку и работает с файлами внутри нее.

In [14]:
class MyCustomFooHandler(HandlerBaseClass):
    """
    FOO file handler.
    """

    bar = None
    baz = None

    @staticmethod
    def extract_text(path: Path) -> Path:
        ...

В этом сценарии метод `extract_text` __не должен__ быть декорирован `@HandlerBaseClass.write_batches`.
***

Новый обработчик можно параметризировать, если это необходимо:

In [15]:
handler_kwargs = {
    ImgTesseractHandler: {...},
    MyCustomFooHandler: {'bar': 1, 'baz': 'two'}
}

Фактическая параметризация обработчика (в случае `MyCustomFooHandler` — обновление атрибутов класса `bar` и `baz`) происходит в момент инициализации сканера `DirectoryScanner`, поскольку словарь `handler_kwargs` передается в `DirectoryScanner` в качестве аргумента.

Наконец, новый обработчик необходимо назначить на расширение `foo` (или _зарегистрировать_ расширение `foo` с помощью обработчика `MyCustomFooHandler`):

In [16]:
DirectoryScanner.update_map({'foo': MyCustomFooHandler})
DirectoryScanner.map

{
    "7z": "ArchiveHandler",
    "csv": "AnyTextFileHandler",
    "doc": "DocxHandler",
    "docx": "DocxHandler",
    "foo": "MyCustomFooHandler",
    "jpeg": "ImgTesseractHandler",
    "jpg": "ImgTesseractHandler",
    "json": "AnyTextFileHandler",
    "msg": "MsgHandler",
    "pdf": "PdfHandler",
    "png": "ImgTesseractHandler",
    "pptx": "PptxHandler",
    "py": "AnyTextFileHandler",
    "rtf": "RtfHandler",
    "txt": "AnyTextFileHandler",
    "xls": "XlsHandler",
    "xlsx": "XlsxHandler",
    "zip": "ZipHandler",
}

Исходное состояние `DirectoryScanner.map` восстанавливается методом класса `DirectoryScanner.restore_map_defaults`:

In [17]:
DirectoryScanner.restore_map_defaults()

# ушли пары 'foo': 'MyCustomFooHandler' и 'jpeg': 'ImgTesseractHandler'
DirectoryScanner.map

{
    "7z": "ArchiveHandler",
    "csv": "AnyTextFileHandler",
    "doc": "DocxHandler",
    "docx": "DocxHandler",
    "jpg": "ImgTesseractHandler",
    "json": "AnyTextFileHandler",
    "msg": "MsgHandler",
    "pdf": "PdfHandler",
    "png": "ImgTesseractHandler",
    "pptx": "PptxHandler",
    "py": "AnyTextFileHandler",
    "rtf": "RtfHandler",
    "txt": "AnyTextFileHandler",
    "xls": "XlsHandler",
    "xlsx": "XlsxHandler",
    "zip": "ZipHandler",
}

Обратите внимание, что это влияет только на соответствие между расширением и обработчиком и не влияет на примененную параметризацию обработчика, например:

In [18]:
# по-прежнему 'rus'
ImgTesseractHandler.tesseract_lang

'rus'

***
### <a id='toc1_1_9_'></a>[__Стратегия резки текста__](#toc0_)

Стратегия резки текста обязана быть потомком базового абстрактного класса `BatchingStrategyBaseClass` и переопределять 2 метода экземпляра, `split` и `get_len`, и свойство `batch_max_size`. Разберем их на примере построчной стратегии `LinewiseBatchingStrategy`, объект которой является значением по умолчанию аргумента `batch_strategy` при инициализации сканера `DirectoryScanner`.

In [19]:
import re

In [20]:
class LinewiseBatchingStrategy(BatchingStrategyBaseClass):
    """
    Linewise text batching strategy.
    """

    batch_max_size = 1000
    split_pattern = re.compile(r'\n+')
    sub_pattern = re.compile(r'(?a)\s+')

    def split(self, text: str) -> list[str]:
        return self.split_pattern.split(text)

    def get_len(self, token: str) -> int:
        return len(self.sub_pattern.sub('', token))

* Метод `split` определяет, каким образом исходный текст разрезается на токены. В случае `LinewiseBatchingStrategy` — по одному или более символу новой строки `\n`.

* Метод `get_len` определяет, каким образом рассчитывается _длина токена_. Это не обязательно длина строки. Это — _характеристическая_ длина, а `get_len` задает некое частное правило, по которому каждому образовавшемуся в результате резки токену ставится в соответствие некоторое целое число. В `LinewiseBatchingStrategy` длина токена фактически есть сумма длин непробельных последовательностей в нем.

Объявление этих кастомных логик — прерогатива исследователя и зависит от нужд конкретной практической задачи.

* Атрибут класса `batch_max_size` определяет максимальный размер батча в единицах длины токена: батч набирается из токенов до тех пор, пока данный параметр не будет превышен. Далее токены батча склеиваются разделителем `\n`. При этом если максимальный размер батча превышается единственным токеном, то батч будет состоять из этого единственного токена (нестрогий набор).

***
### <a id='toc1_1_10_'></a>[__Писатель отчета__](#toc0_)

Отчет писателя `HtmlReporter`, объект которого является значением по умолчанию аргумента `reporter` при инициализации сканера `DirectoryScanner`, см. в [__Отчет писателя `HtmlReporter`__](#toc1_1_6_). Опишем требования к произвольному писателю.

Писатель отчета обязан быть потомком абстрактного класса `ReporterBaseClass` и переопределять метод `write`, который реализует запись на диск атрибута сканера `DirectoryScanner.records`. Описание структуры `DirectoryScanner.records` см. в [__Публичный интерфейс класса `DirectoryScanner`__](#toc1_1_7_).

In [21]:
class MyCustomReporter(ReporterBaseClass):
    """
    My custom report writer.
    """

    def write(self, records_dict):
        """Write report with `DirectoryScanner.records` to disk. """
        ...